# Run Hugging Face flan-t5-xxl
`  sampling on Inf2

In [1]:
# !pip install git+https://github.com/aws-neuron/transformers-neuronx.git transformers -U
# !pip install accelerate
# !pip install SentencePiece

## Download and construct the model

We download and construct the `facebook/opt-13b` model using the Hugging Face `from_pretrained` method.

In [2]:
import torch
import torch_neuronx
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSequenceClassification

In [4]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl")

In [6]:
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xxl", device_map="auto")


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [21]:
# CPU output

In [12]:
input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

<pad> Wie alt sind Sie?</s>


In [13]:
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('xla')

In [14]:
model_neuron = torch_neuronx.trace(model.generate, input_ids)

2023-05-21T08:37:35Z WARNING 412501 [py.warnings]: /opt/aws_neuron_venv_pytorch/bin/neuronx-cc:8: RuntimeWarning: overflow encountered in int_scalars
  sys.exit(main())

2023-05-21T08:37:35Z WARNING 412501 [LayoutBottleneck]: Connected component _subtract.1 has no matmult/reduce/batchnorm. Guessing layout. Considering putting on CPU.


In [15]:
filename = 'model_flan-t5-xxl.pt'
torch.jit.save(model_neuron, filename)

In [53]:
model_neuron_load = torch.jit.load(filename) 
neuron_outputs=model_neuron_load(input_ids)
print(tokenizer.decode(neuron_outputs[0]))

<pad> Wie alt sind Sie?</s>


In [54]:
def parameters_calc(model, flag):
    parameter = sum(p.numel() for p in model.parameters())
    multipliers = {'K':10**3, 'M':10**6, 'B':10**9}
    
    mult = int(multipliers[flag]) # look up suffix to get multiplier
     # convert number to float, multiply by multiplier, then make int
    return str(round(float(parameter) / mult, 2)) + flag


print(parameters_calc(model, 'B'))

11.14B
